# Image Classification with the MNIST Dataset

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms
import random
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,30)
        self.fc2=nn.Linear(30,10)
    def forward(self,x):
        l1=self.fc1(x)
        al1=torch.sigmoid(l1)
        l2=self.fc2(al1)
        al2=torch.sigmoid(l2)
        return al2

In [3]:
def train(model,use_cuda,train_loader,optimizer,epoch):
    model.train()
    for batchid,(data,target) in enumerate(train_loader):
        y_onehot = torch.zeros([target.shape[0], 10])  # Zero vector of shape [batch_size, 10]
        y_onehot[range(target.shape[0]), target] = 1

        data = data.view([data.shape[0], 784]) # [batch_size,784]
        if use_cuda:
            data,y_onehot=data.cuda(),y_onehot.cuda()
        optimizer.zero_grad()
        output=model(data)
        loss=torch.mean((output-y_onehot)**2)
        loss.backward()
        optimizer.step()
        if batchid % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batchid * len(data), len(train_loader.dataset),
            100. * batchid / len(train_loader), loss.item()))

In [4]:
def test(model,use_cuda,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data=data.reshape([data.shape[0],784])
            y_onehot=torch.zeros([target.shape[0],10])
            y_onehot[range(target.shape[0]),target]=1
            if use_cuda:
                data,y_onehot=data.cuda(),y_onehot.cuda()
            output=model(data)
            test_loss+=torch.sum((output-y_onehot)**2)
            pred=output.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item() 
        test_loss/=len(test_loader.dataset)
        print(test_loss,100*correct/len(test_loader.dataset))

In [5]:
def seed(seed_value):
    torch.cuda.manual_seed_all(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    torch.backends.cudnn.benchmark=False
    torch.backends.cudnn.deterministic=True

In [14]:
def main():
    use_cuda=False
    seed(0)
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ])#normalizing and transforming data to tensor
    data1=datasets.MNIST('',download=True,train=True,transform=transform)
    data2=datasets.MNIST('',train=False,transform=transform)
    #loading data using torch's inbuilt dataloader
    train_loader=torch.utils.data.DataLoader(data1,num_workers=0,batch_size=64,shuffle=True)
    test_loader=torch.utils.data.DataLoader(data2,num_workers=0,batch_size=1000,shuffle=False)
    model=Net()
    if use_cuda:
        model=model.cuda()
    optimizer=optim.SGD(model.parameters(),lr=10)
    for epoch in range(1,11):
        train(model,use_cuda,train_loader,optimizer,epoch)
        test(model,use_cuda,test_loader)
    torch.save(model.state_dict(),"mnist_cnn.pt")  #saving the model
    model.load_state_dict(torch.load("mnist_cnn.pt"))

In [15]:
if __name__=="__main__":
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.284883
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.033432
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.021424
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.014365
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.013061
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.012427
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.015567
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.019503
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.025899
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.012034
tensor(0.1259) 92.86
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.015895
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.007555
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.015096
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.018973
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.015171
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.012200
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.014843
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.014440
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.008070
Train Epoch: 2 [57600/60000 (96%)]	Loss: 